In [23]:
import json
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import shutil
import matplotlib.pyplot as plt
import time
import numpy as np
from api import google

In [24]:
!mkdir shapefile
df = pd.read_csv('location.csv')
df.dropna(inplace=True, how='all')
amount = len(df)

mkdir: cannot create directory ‘shapefile’: File exists


In [25]:
# add four points in the north, east, south and west direction 500m away from original one
for i in range(amount):
    j = 1
    while j <= 4:
        df.loc[i+(amount*j)] = df.loc[i]
        j += 1
df = df.sort_values(by=['Name'])
df = df.reset_index()
del df['index']

for i in range(amount):
    i = i*5
    df.loc[i, 'Lat'] = df.loc[i+4].Lat + (0.5/111*np.cos(df.loc[i+4, 'Lat']))
    df.loc[i+1, 'Lon'] = df.loc[i+4].Lon + (0.5/111)
    df.loc[i+2, 'Lat'] = df.loc[i+4].Lat - (0.5/111*np.cos(df.loc[i+4, 'Lat']))
    df.loc[i+3, 'Lon'] = df.loc[i+4].Lon - (0.5/111)

In [26]:
df

,Name,Lat,Lon
0,Boston,42.347501,-71.045869
1,Boston,42.347787,-71.041364
2,Boston,42.348074,-71.045869
3,Boston,42.347787,-71.050373
4,Boston,42.347787,-71.045869
5,Cambridge,42.365427,-71.103839
6,Cambridge,42.365634,-71.099334
7,Cambridge,42.365841,-71.103839
8,Cambridge,42.365634,-71.108344
9,Cambridge,42.365634,-71.103839


In [27]:

key = google
category = ['art_gallery',
            'amusement_park',
            'bakery', 
            'bar', 
            'book_store', 
            'cafe', 
            'gym', 
            'library', 
            'movie_theater', 
            'museum', 
            'night_club', 
            'park', 
            'restaurant', 
            'school', 
            'shopping_mall', 
            'spa', 
            'stadium']  
# category = ['art_gallery']

In [33]:
df['Name'].unique()

array(['Boston', 'Cambridge', 'London', 'Manhattan', 'San_Francisco',
       'Seattle', 'Silcon_Valley'], dtype=object)

In [34]:
lat = list(df['Lat'])
lng = list(df['Lon'])
city = list(df['Name'])

all_info = pd.DataFrame(columns=df['Name'].unique())
all_info['type'] = category
    
for x,i in enumerate(category):
    for j in range(len(lat)):
        
        if j > 0:
        
            if city[j] == city[j-1]: 
                pass
            else: 
                lat_info = []
                lng_info = []
        else:
            lat_info = []
            lng_info = []

        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='+str(lat[j])+','+str(lng[j])+'&radius=500&type='+str(i)+'&key='+key
        data = requests.get(url)
        text = json.loads(data.text)

        for k in range(len(text['results'])):

            lat_info.append(text['results'][k]['geometry']['location']['lat'])

            lng_info.append(text['results'][k]['geometry']['location']['lng'])
        
        time.sleep(5) 
        
        if 'next_page_token' not in text:
            print('only 1 page')

        else:
            url2 = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken='+text['next_page_token']+'&key='+key
            data2 = requests.get(url2)
            text2 = json.loads(data2.text)
            for k in range(len(text2['results'])):
                lat_info.append(text2['results'][k]['geometry']['location']['lat'])
                lng_info.append(text2['results'][k]['geometry']['location']['lng'])
            
            time.sleep(5)

            if 'next_page_token' not in text2:
                print('only 2 page')
            else:

                next_page_token2 = text2['next_page_token']
                url3 = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken='+text2['next_page_token']+'&key='+key
                data3 = requests.get(url3)
                text3 = json.loads(data3.text)
                for k in range(len(text3['results'])):
                    lat_info.append(text3['results'][k]['geometry']['location']['lat'])
                    lng_info.append(text3['results'][k]['geometry']['location']['lng'])

        info = pd.DataFrame()
        info['lat'] = lat_info
        info['lng'] = lng_info
        info.drop_duplicates(inplace=True)
        geometry = [Point(xy) for xy in zip(info.lng, info.lat)]
        crs = {'init': 'epsg:4326'}
        info_geo = gpd.GeoDataFrame(info, crs=crs, geometry=geometry)
        locals()[str(city[j])+'_'+str(i)] = info_geo
        if len(info_geo) > 0:
            info_geo.to_file('shapefile/%s_%s.shp'%(city[j], i), driver='ESRI Shapefile')
        all_info['%s'%city[j]][x] = len(info_geo)
        print(str(city[j]), j, str(i), 'done')
    

only 1 page
Boston 0 art_gallery done


/nfshome/mh5172/.conda/envs/mingyipy36/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


only 1 page
Boston 1 art_gallery done
only 1 page
Boston 2 art_gallery done
only 1 page
Boston 3 art_gallery done
only 1 page
Boston 4 art_gallery done
only 1 page
Cambridge 5 art_gallery done
only 1 page
Cambridge 6 art_gallery done
only 1 page
Cambridge 7 art_gallery done
only 1 page
Cambridge 8 art_gallery done
only 1 page
Cambridge 9 art_gallery done
only 1 page
London 10 art_gallery done
only 1 page
London 11 art_gallery done
only 1 page
London 12 art_gallery done
only 1 page
London 13 art_gallery done
only 1 page
London 14 art_gallery done
Manhattan 15 art_gallery done
Manhattan 16 art_gallery done
Manhattan 17 art_gallery done
Manhattan 18 art_gallery done
Manhattan 19 art_gallery done
San_Francisco 20 art_gallery done
San_Francisco 21 art_gallery done
only 1 page
San_Francisco 22 art_gallery done
San_Francisco 23 art_gallery done
San_Francisco 24 art_gallery done
only 1 page
Seattle 25 art_gallery done
only 1 page
Seattle 26 art_gallery done
only 1 page
Seattle 27 art_gallery d

only 2 page
San_Francisco 21 gym done
only 1 page
San_Francisco 22 gym done
only 1 page
San_Francisco 23 gym done
only 2 page
San_Francisco 24 gym done
only 1 page
Seattle 25 gym done
only 1 page
Seattle 26 gym done
only 1 page
Seattle 27 gym done
only 1 page
Seattle 28 gym done
only 1 page
Seattle 29 gym done
only 1 page
Silcon_Valley 30 gym done
only 1 page
Silcon_Valley 31 gym done
only 1 page
Silcon_Valley 32 gym done
only 1 page
Silcon_Valley 33 gym done
only 1 page
Silcon_Valley 34 gym done
only 1 page
Boston 0 library done
only 1 page
Boston 1 library done
only 1 page
Boston 2 library done
only 1 page
Boston 3 library done
only 1 page
Boston 4 library done
only 1 page
Cambridge 5 library done
only 1 page
Cambridge 6 library done
only 1 page
Cambridge 7 library done
only 1 page
Cambridge 8 library done
only 1 page
Cambridge 9 library done
only 1 page
London 10 library done
only 1 page
London 11 library done
only 1 page
London 12 library done
only 1 page
London 13 library done
onl

only 2 page
Cambridge 6 school done
only 2 page
Cambridge 7 school done
only 2 page
Cambridge 8 school done
only 2 page
Cambridge 9 school done
only 1 page
London 10 school done
only 1 page
London 11 school done
only 1 page
London 12 school done
only 2 page
London 13 school done
only 1 page
London 14 school done
Manhattan 15 school done
Manhattan 16 school done
Manhattan 17 school done
Manhattan 18 school done
Manhattan 19 school done
San_Francisco 20 school done
only 2 page
San_Francisco 21 school done
only 2 page
San_Francisco 22 school done
only 2 page
San_Francisco 23 school done
San_Francisco 24 school done
only 1 page
Seattle 25 school done
only 1 page
Seattle 26 school done
only 1 page
Seattle 27 school done
only 1 page
Seattle 28 school done
only 1 page
Seattle 29 school done
only 1 page
Silcon_Valley 30 school done
only 1 page
Silcon_Valley 31 school done
only 1 page
Silcon_Valley 32 school done
only 1 page
Silcon_Valley 33 school done
only 1 page
Silcon_Valley 34 school done


In [35]:
shutil.make_archive('shp_0108', 'zip', 'shapefile')
all_info.to_csv('count_0108.csv', sep=',')